# Training the LSTM on your Data

This notebook will take you through the steps necessary to train an LSTM to recognize ICD-9 codes, or items from similar dictionaries, from free text.

## Setup

### Imports

Make sure that the below packages are installed on the server on which this program will run.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
import tensorflow as tf
from simpleLSTMWithNNetModel import Model, reloadModel
from trainModel import trainModel
import os
import pprint
import numpy as np
import pickle
pp = pprint.PrettyPrinter(indent=4)

### Directories

Executing the following cell will prompt the user to type in the directories corresponding to the training and validation sets, the vocabulary, and the word vector mappings. Defaults are given in the comments below.

Note that the training and test data needs to have one column dedicated to free text (`noteIdx`) and another dedicated to top-level ICD-9 codes (`codeIdx`) associated with each patient. Preferably, the latter should be strung together using '-' as the delimiter (e.g. for patient 1, 1-2-6-4).

Please make sure that the parameters such as the embed size, maximum note length, learning rate, number of maximum training epochs, batch size, hidden layer size, number of neural net layers, and probabilities are to your specification.

### Put sample file with require file headers:

adjust training data headers to match small icd9 training file

In [3]:
data_train = raw_input('What is the path to the training data? ') #default: data/icd9NotesDataTable_train.csv
data_valid = raw_input('What is the path to the validation data? ') #default: data/icd9NotesDataTable_valid.csv
vocab = raw_input('What is the path to the vocabulary? ') #default: data/icd9Vocab.txt
wordVecs = raw_input('What is the path to the vocabulary? ') #data/newgloveicd9.txt. These are length 300 word vectors from GloVE

NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 300 # this should correspond to the length of the word vectors
maxAllowedNoteLength = 1000
max_grad_norm = 5
codeIdx = raw_input('Which column contains top-level ICD-9 codes (outputs) in the training and test data? Default is 9. ')
textIdx = raw_input('Which column contains notes/text (inputs) in the training and test data? Default is 6. ')
learning_rate = 0.001
training_epochs = 100
batch_size = 256
n_hidden = 200
output_keep_prob = 0.5
input_keep_prob = 1
numLayers = 1

What is the path to the training data? data/0815/mimic_0815_train.csv
What is the path to the validation data? data/0815/mimic_0815_valid.csv
What is the path to the vocabulary? data/icd9Vocab.txt
What is the path to the vocabulary? data/newgloveicd9.txt
Which column contains top-level ICD-9 codes (outputs) in the training and test data? Default is 9. 9
Which column contains notes/text (inputs) in the training and test data? Default is 6. 6


In [4]:
# NOTE:
# Fixing some issue ashley had

textIdx = int(textIdx)
codeIdx = int(codeIdx)# I'm not sure how models were trinaed before if this part was broken

Here, make sure that the output path is specified as you would like. By default, the program saves the output in a folder with the name of your choice within the folder `results`.

If there exists a folder with results that you would like to load again, use that here.

In [5]:
output_path = raw_input('Where are models and performances (to be) saved? ')
output_path = os.path.join('results', output_path)
if output_path == 'results/':
    output_path = 'results/temp'
if not os.path.exists(output_path):
    os.makedirs(output_path)

Where are models and performances (to be) saved? 0815_mimic


## Initialization

Executing the following cell will ask whether or not there is a previously saved model; if not, the model will train features from scratch, and if so, the features will be loaded.

Note that AZ added "int() to the codeIdx and textIdx to resolve some errors that were preventing it from initializing

In [7]:
sizeList = [n_hidden, 150, 75] # these are the weights we will be using

def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is True for "yes" or False for "no".
    """
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")
            
prev_model = query_yes_no("Is there a previously trained model?")

if prev_model:
    helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = int(codeIdx), textIdx = int(textIdx),
    helperLoadPath = output_path)
else:
    #print codeIdx
    #print textIdx
    helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = int(codeIdx), textIdx = int(textIdx))
    
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
helper.save(output_path) # token2id and max length saved to output_path
sizeList.append(helper.n_labels)

total_batches = (xTrain.shape[0]//batch_size)
print('Total number of batches per epoch: %d'%(total_batches))
print('Maximum note length: %d'%(helper.max_length))
print('Number of ICD-9 codes: %d'%(helper.n_labels))
print('There are a total of: {} ICD-9 codes'.format(len(helper.icdDict.keys())))
pp.pprint(helper.icdDict)
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec length = %d'%(embeddings.shape))

Is there a previously trained model? [Y/n] m
Please respond with 'yes' or 'no' (or 'y' or 'n').
Is there a previously trained model? [Y/n] m
Please respond with 'yes' or 'no' (or 'y' or 'n').
Is there a previously trained model? [Y/n] n
Total number of batches per epoch: 154
Maximum note length: 1000
Number of ICD-9 codes: 19
There are a total of: 19 ICD-9 codes
{   'cat:1': 8,
    'cat:10': 16,
    'cat:11': 15,
    'cat:12': 14,
    'cat:13': 13,
    'cat:14': 12,
    'cat:15': 11,
    'cat:16': 10,
    'cat:17': 9,
    'cat:18': 18,
    'cat:19': 17,
    'cat:2': 7,
    'cat:3': 6,
    'cat:4': 5,
    'cat:5': 4,
    'cat:6': 3,
    'cat:7': 2,
    'cat:8': 1,
    'cat:9': 0}
xDev shape: nObs = 13181, nWords = 1000
yDev shape: nObs = 13181, nClasses = 19
xTrain shape: nObs = 39541, nWords = 1000
yTrain shape: nObs = 39541, nClasses = 19
Embeddings shape: nWords = 10008, wordVec length = 300


The following cell initializes the dictionary of hyperparameters for the model that fully describe the model.

In [8]:
hyperParamDict = {'EMBED_SIZE': EMBED_SIZE,
                  'maxNoteLength': maxAllowedNoteLength,
                  'maxGradNorm': max_grad_norm,
                  'outputKeepProb': output_keep_prob,
                  'inputKeepProb': input_keep_prob,
                  'learningRate': learning_rate,
                  'trainingEpochsMax': training_epochs,
                  'batchSize': batch_size,
                  'n_hidden': n_hidden,
                 'numLayers': numLayers,
                 'sizeList':sizeList}
pp.pprint(hyperParamDict)
with open(os.path.join(output_path, 'hyperParamDict.pickle'), 'wb') as handle:
    pickle.dump(hyperParamDict, handle, protocol = 2)
    #dumping with 2 because ALTUD uses python 2.7 right now.

{   'EMBED_SIZE': 300,
    'batchSize': 256,
    'inputKeepProb': 1,
    'learningRate': 0.001,
    'maxGradNorm': 5,
    'maxNoteLength': 1000,
    'n_hidden': 200,
    'numLayers': 1,
    'outputKeepProb': 0.5,
    'sizeList': [200, 150, 75, 19],
    'trainingEpochsMax': 100}


## Training

Finally, the model is trained (be wary - it will take some time; on an Amazon Deep Learning AMI, it took around an hour to train)...

In [9]:
from trainModel import trainModel
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
trainModel(helperObj = helper, embeddings = embeddings, hyperParamDict = hyperParamDict, 
          xDev = xDev, xTrain = xTrain, yDev = yDev, yTrain = yTrain, 
           lastTrueWordIdx_dev = lastTrueWordIdx_dev, 
           lastTrueWordIdx_train = lastTrueWordIdx_train,
           training_epochs = training_epochs, 
           output_path = output_path, batchSizeTrain = batch_size,
           sizeList = sizeList,
           maxIncreasingLossCount = 100, batchSizeDev = 1500, chatty = True)

shape of embeddings
(?, 1000, 300)
<class 'tensorflow.python.ops.rnn_cell_impl.MultiRNNCell'>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1000, 300)
cell output size
200
cell state size
(LSTMStateTuple(c=200, h=200),)
output shape
(?, 1000, 200)
offset shape
(?, 1)
output shape new shape
(?, 200)
flattened indices shape
(?, 1)
output flattened shape
(?, 200)
(200, 150)
W_1 shape
(150,)
b_1 shape
(150, 75)
W_2 shape
(75,)
bias shape
(19,)
U shape
(75, 19)
bias shape
(19,)
output wx + b
(?, 19)
(?, 19)


/usr/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


***************************
***************************
Running on epoch 0
***************************
***************************
running iteration 0 with loss 0.712959 at time 2.177852
running iteration 25 with loss 0.504476 at time 33.703794
running iteration 50 with loss 0.492231 at time 65.270715
running iteration 75 with loss 0.486564 at time 96.834131
running iteration 100 with loss 0.485935 at time 128.420802
running iteration 125 with loss 0.487968 at time 160.004188
running iteration 150 with loss 0.479473 at time 191.595145
average training loss 0.499373
test loss 0.495483
Total run time was 211.958379
New best model found. Saving
results/0815_mimic
***************************
***************************
Running on epoch 1
***************************
***************************
running iteration 0 with loss 0.471719 at time 1.266999
running iteration 25 with loss 0.460605 at time 32.876569
running iteration 50 with loss 0.479632 at time 64.481843
running iteration 75 with lo

running iteration 25 with loss 0.382165 at time 32.914015
running iteration 50 with loss 0.384149 at time 64.524196
running iteration 75 with loss 0.380359 at time 96.147809
running iteration 100 with loss 0.376495 at time 127.761305
running iteration 125 with loss 0.366010 at time 159.374370
running iteration 150 with loss 0.370385 at time 191.002553
average training loss 0.375875
test loss 0.430305
Total run time was 211.499705
New best model found. Saving
results/0815_mimic
***************************
***************************
Running on epoch 13
***************************
***************************
running iteration 0 with loss 0.360684 at time 1.274921
running iteration 25 with loss 0.376052 at time 32.906664
running iteration 50 with loss 0.375370 at time 64.528341
running iteration 75 with loss 0.373947 at time 96.153116
running iteration 100 with loss 0.366902 at time 127.774572
running iteration 125 with loss 0.358759 at time 159.396725
running iteration 150 with loss 0.36

and the session closed. You should be able to see your results in the `output_path` directory you specified earlier.

To evaluate the results and generate plots and such, please check out `predictionEvaluation.ipynb` in the same repository.

In [10]:
#FOR USE IN LOADING OLD MODELS AND REEVALUATING
# xDev[xDev == -1] = 0
# xTrain[xTrain == -1] = 0
# trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))
# with tf.Session() as session:
#     tf.global_variables_initializer().run()
#     modelDict = reloadModel(session = session,
#                             saverCheckPointPath = output_path,
#                             saverMetaPath = output_path + '/bestModel.meta')
#     print('here we go')
#     for i in range(3):
#         pred_y = session.run(modelDict['y_last'],feed_dict={modelDict['xPlaceHolder']: xDev,
#                                       modelDict['trueWordIdxs']:lastTrueWordIdx_dev,
#                                       modelDict['outputKeepProb']: 1.0,
#                                       modelDict['inputKeepProb']: 1.0}, ) 
#         validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
#                                              labels = tf.cast(yDev, tf.float32))
#         validLoss = tf.reduce_mean(validLoss)
#         validLoss = validLoss.eval()
#         print('test loss %f'%(validLoss))
#         print('***********************************************')

In [11]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()